In [1]:
import numpy as np
import scipy.stats
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
AAPL_df=pd.read_csv('AAPL.csv',index_col=0)
TSLA_df=pd.read_csv('TSLA.csv',index_col=0)
TXN_df=pd.read_csv('TXN.csv',index_col=0)
GME_df=pd.read_csv('GME.csv',index_col=0)
GM_df=pd.read_csv('GM.csv',index_col=0)

In [14]:
AAPL_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-03-23,35.314999,35.395000,35.152500,35.230000,33.262756,81385200
2017-03-24,35.375000,35.435001,35.087502,35.160000,33.196659,89582400
2017-03-27,34.847500,35.305000,34.654999,35.220001,33.253319,94300400
2017-03-28,35.227501,36.009998,35.154999,35.950001,33.942547,133499200
2017-03-29,35.919998,36.122501,35.797501,36.029999,34.018082,116760000


In [15]:
AAPL=AAPL_df['Adj Close'].pct_change().dropna()
TSLA=TSLA_df['Adj Close'].pct_change().dropna()
GME=GME_df['Adj Close'].pct_change().dropna()
GM=GM_df['Adj Close'].pct_change().dropna()
TXN=TXN_df['Adj Close'].pct_change().dropna()
#AAPL= AAPL_df.loc[:,'Adj Close']
#TSLA= TSLA_df.loc[:,'Adj Close']
#GME= GME_df.loc[:,'Adj Close']
#GM= GM_df.loc[:,'Adj Close']
#TXN= TXN_df.loc[:,'Adj Close']

stock_list = [AAPL_df, TSLA_df, GME_df, GM_df, TXN_df]
'''for stock in stock_list:
    stock=stock['Adj Close'].pct_change()'''
AAPL

Date
2017-03-24   -0.001987
2017-03-27    0.001707
2017-03-28    0.020727
2017-03-29    0.002225
2017-03-30   -0.001318
                ...   
2022-03-16    0.029015
2022-03-17    0.006454
2022-03-18    0.020919
2022-03-21    0.008538
2022-03-22    0.020801
Name: Adj Close, Length: 1258, dtype: float64

In [5]:

stock_list = [AAPL, TSLA, GME, GM, TXN]
df_stock = pd.concat(stock_list,axis=1)
df_stock.columns = ['AAPL', 'TSLA', 'GME', 'GM', 'TXN']
df_stock = df_stock.sort_index(ascending=True)
df_stock.head()

,AAPL,TSLA,GME,GM,TXN
Date,,,,,
2017-03-24,-0.001987,NaN,-0.136060,0.008757,-0.000992
2017-03-27,0.001707,NaN,0.007247,0.004340,-0.000745
2017-03-28,0.020727,NaN,0.041726,0.024489,0.000994
2017-03-29,0.002225,NaN,0.010589,-0.000562,-0.001489
2017-03-30,-0.001318,NaN,0.023235,-0.003658,0.006833


In [6]:
#quantify the fama french 5 factor
df_ff=pd.read_csv('FF_Research_Data_5_Factors.csv',index_col=0)
df_ff.head()
#since the fama french has only integer datetime, so we need to align their date format
#try millions time..
df_ff.index=pd.to_datetime(df_ff.index, format = '%Y%m%d').strftime('%Y-%m-%d')
df_ff.head()


,Mkt-RF,SMB,HML,RMW,CMA,RF
1963-07-01,-0.67,0.01,-0.35,0.03,0.11,0.012
1963-07-02,0.79,-0.31,0.24,-0.08,-0.25,0.012
1963-07-03,0.63,-0.16,-0.09,0.13,-0.24,0.012
1963-07-05,0.40,0.09,-0.26,0.07,-0.28,0.012
1963-07-08,-0.63,0.07,-0.19,-0.27,0.06,0.012


In [7]:
#merge two return to do the linear regression
df = pd.merge(df_stock, df_ff, left_index=True, right_index=True, how='inner')
df = df.fillna(0)
#rf use avg of 2019-2022 avg(2.14%,0.04%0.44%)
rf = ((1.0214+1.00004+1.0044)/3) ** (1/360) - 1
df = df - rf
df2 = df.copy()
df = df['2019-01-02':][:250]
#df.shape 250 rolling window
#df.head()

In [8]:
import statsmodels.api as sm

stock_names = {
    'AAPL': 'Aaple',
    'TSLA': 'Tesla',
    'GME': 'GME',
    'GM': 'GM',
    'TXN': 'Texas Instrument'
}
#fit the multi linear regression
'''for stock in ['AAPL', 'TSLA', 'GME', 'GM', 'TXN']:
    model = sm.OLS(df[stock], sm.add_constant(
        df[['Mkt-RF','SMB', 'HML','RMW','CMA']].values))
    result = model.fit()
    print(stock_names[stock] + '\n')
    print(result.summary())
    #CHECK BETA EXPLANATORY POWER AND R-SQUARE
    print('\n\n')
    print('\n\n')'''
#AAPL'S RESULT
X=df[['Mkt-RF','SMB', 'HML','RMW','CMA']]
Y=df['AAPL']
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)


                                 OLS Regression Results                                
Dep. Variable:                   AAPL   R-squared (uncentered):                   0.596
Model:                            OLS   Adj. R-squared (uncentered):              0.588
Method:                 Least Squares   F-statistic:                              72.27
Date:                Thu, 31 Mar 2022   Prob (F-statistic):                    2.97e-46
Time:                        20:50:30   Log-Likelihood:                          781.63
No. Observations:                 250   AIC:                                     -1553.
Df Residuals:                     245   BIC:                                     -1536.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [9]:
#TSLA'S RESULT
X=df[['Mkt-RF','SMB', 'HML','RMW','CMA']]
Y=df['TSLA']
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                   TSLA   R-squared (uncentered):                   0.153
Model:                            OLS   Adj. R-squared (uncentered):              0.136
Method:                 Least Squares   F-statistic:                              8.843
Date:                Thu, 31 Mar 2022   Prob (F-statistic):                    9.86e-08
Time:                        20:50:30   Log-Likelihood:                          536.31
No. Observations:                 250   AIC:                                     -1063.
Df Residuals:                     245   BIC:                                     -1045.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [10]:
#GME'S RESULT
X=df[['Mkt-RF','SMB', 'HML','RMW','CMA']]
Y=df['GME']
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                    GME   R-squared (uncentered):                   0.169
Model:                            OLS   Adj. R-squared (uncentered):              0.152
Method:                 Least Squares   F-statistic:                              9.961
Date:                Thu, 31 Mar 2022   Prob (F-statistic):                    1.09e-08
Time:                        20:50:30   Log-Likelihood:                          442.49
No. Observations:                 250   AIC:                                     -875.0
Df Residuals:                     245   BIC:                                     -857.4
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [11]:
#GM'S RESULT
X=df[['Mkt-RF','SMB', 'HML','RMW','CMA']]
Y=df['GM']
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                     GM   R-squared (uncentered):                   0.462
Model:                            OLS   Adj. R-squared (uncentered):              0.451
Method:                 Least Squares   F-statistic:                              42.11
Date:                Thu, 31 Mar 2022   Prob (F-statistic):                    3.33e-31
Time:                        20:50:30   Log-Likelihood:                          765.65
No. Observations:                 250   AIC:                                     -1521.
Df Residuals:                     245   BIC:                                     -1504.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [12]:
#TXN'S RESULT
X=df[['Mkt-RF','SMB', 'HML','RMW','CMA']]
Y=df['TXN']
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                    TXN   R-squared (uncentered):                   0.474
Model:                            OLS   Adj. R-squared (uncentered):              0.463
Method:                 Least Squares   F-statistic:                              44.10
Date:                Thu, 31 Mar 2022   Prob (F-statistic):                    2.43e-32
Time:                        20:50:30   Log-Likelihood:                          744.84
No. Observations:                 250   AIC:                                     -1480.
Df Residuals:                     245   BIC:                                     -1462.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------